# MICS Data Preprocessing

This notebook preprocesses the raw MICS data for DoubleML analysis:
- Defines outcome variables (Y), treatment variable (T), and covariates (X)
- Encodes ordinal variables with proper mappings
- One-hot encodes categorical variables
- Exports cleaned data (NaN values preserved for later handling)

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import OneHotEncoder

## 1. Load Raw Data

In [2]:
# Load raw data
mics = pd.read_csv("mics.csv", low_memory=False)

print(f"Dataset shape: {mics.shape}")
mics.head()

Dataset shape: (56721, 785)


,HH1,HH2,HINT,HH3,HH4,HH5D,HH5M,HH5Y,HH6,HH7,...,RiskHome_0_12,RiskSource_0_12,water_treatment3,Any_U5,Region,windex_ur,windex5_categ,helevel_temp,wq27_decile,SomeRiskHome
0,1,5,12.0,12,11,2,6. JUNE,2017,2. Rural,1. EAST,...,1,1,0,1,1,2,Poor,NaN,7,1
1,1,14,15.0,15,11,3,6. JUNE,2017,2. Rural,1. EAST,...,1,0,0,1,1,2,Poor,NaN,1,1
2,1,22,15.0,15,11,4,6. JUNE,2017,2. Rural,1. EAST,...,1,1,0,1,1,2,Middle,NaN,8,1
3,2,3,12.0,12,11,5,6. JUNE,2017,2. Rural,1. EAST,...,1,1,0,1,1,2,Middle,NaN,8,1
4,2,11,12.0,12,11,5,6. JUNE,2017,2. Rural,1. EAST,...,1,1,0,0,1,1,Poor,NaN,8,1


## 2. Define Variable Groups

In [3]:
# Outcome variables (Y)
Y_cols = ["SomeRiskHome", "VeryHighRiskHome"]

# Treatment variable (T)
T_cols = ["water_treatment"]

# Binary covariates
X_binary_cols = [
    "Any_U5", "Girls_less_than15", "Boys_15or_less",
    "ImprovedWaterSource", "PipedWater", "WellandSpringWater",
    "RainandSurfaceWater", "PurchasedWater",
    "Basic_water_service", "Limited_water_service", "Unimproved_water_service",
    "improved_latrine", "Flush", "Pit_latrine", "Open_defecation",
    "rainy_season",
]

# Ordinal covariates (to be mapped)
X_ordinal_cols_pre_encoding = {
    "helevel": "helevel_ord",
    "windex5": "windex5_ord",
    "RiskSource": "RiskSource_ord",
    "water_carrier_edu": "water_carrier_edu_ord",
}

# Categorical covariates (to be one-hot encoded)
X_categorical_cols = ["Region", "country_cat", "WS1_g"]

print(f"Outcome variables: {Y_cols}")
print(f"Treatment variable: {T_cols}")
print(f"Binary covariates: {len(X_binary_cols)} variables")
print(f"Ordinal covariates: {len(X_ordinal_cols_pre_encoding)} variables")
print(f"Categorical covariates: {len(X_categorical_cols)} variables")

Outcome variables: ['SomeRiskHome', 'VeryHighRiskHome']
Treatment variable: ['water_treatment']
Binary covariates: 16 variables
Ordinal covariates: 4 variables
Categorical covariates: 3 variables


## 3. Define Ordinal Mappings

In [4]:
# Mappings for ordinal variables
ordinal_mappings = {
    "helevel": {"No education": 0, "Primary": 1, "Secondary or higher": 2},
    "windex5": {"Poorest": 0, "Poor": 1, "Middle": 2, "Rich": 3, "Richest": 4},
    "RiskSource": {"No risk": 0, "Moderate to high risk": 1, "Very high risk": 2},
}

print("Ordinal variable mappings:")
for var, mapping in ordinal_mappings.items():
    print(f"\n{var}:")
    for key, val in mapping.items():
        print(f"  {key} -> {val}")

Ordinal variable mappings:

helevel:
  No education -> 0
  Primary -> 1
  Secondary or higher -> 2

windex5:
  Poorest -> 0
  Poor -> 1
  Middle -> 2
  Rich -> 3
  Richest -> 4

RiskSource:
  No risk -> 0
  Moderate to high risk -> 1
  Very high risk -> 2


## 4. Apply Ordinal Mappings

In [5]:
# Apply mappings for ordinal variables, handling NaN
for col, mapping in ordinal_mappings.items():
    if mics[col].dtype == 'object':
        mics[col] = mics[col].map(mapping).astype('Int64')  # Int64 allows NaN
        print(f"Mapped {col}: {mics[col].value_counts().sort_index().to_dict()}")

# Handle water_carrier_edu (98 = missing)
if mics['water_carrier_edu'].dtype != 'Int64':
    mics['water_carrier_edu'] = mics['water_carrier_edu'].replace(98, np.nan).astype('Int64')
    print(f"\nwater_carrier_edu unique values: {sorted(mics['water_carrier_edu'].dropna().unique())}")

Mapped helevel: {np.int64(0): 14336, np.int64(1): 18003, np.int64(2): 24382}
Mapped windex5: {np.int64(0): 14672, np.int64(1): 11911, np.int64(2): 11023, np.int64(3): 10205, np.int64(4): 8910}
Mapped RiskSource: {np.int64(0): 23595, np.int64(1): 21064, np.int64(2): 12062}

water_carrier_edu unique values: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)]


## 5. One-Hot Encode Categorical Variables

In [6]:
# One-hot encode categorical variables
if X_categorical_cols:
    onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')
    encoded_cats = onehot_encoder.fit_transform(mics[X_categorical_cols])
    encoded_df = pd.DataFrame(
        encoded_cats,
        columns=onehot_encoder.get_feature_names_out(X_categorical_cols)
    )
    
    print(f"One-hot encoded {len(X_categorical_cols)} categorical variables into {len(encoded_df.columns)} columns")
    print(f"\nEncoded column names (first 10): {list(encoded_df.columns[:10])}")
    
    # Concatenate with original data and drop original categorical columns
    mics = pd.concat([mics, encoded_df], axis=1)
    mics.drop(X_categorical_cols, axis=1, inplace=True)
    
    print(f"\nDataset shape after encoding: {mics.shape}")

One-hot encoded 3 categorical variables into 32 columns

Encoded column names (first 10): ['Region_2', 'Region_3', 'country_cat_Benin', 'country_cat_Central African Republic', 'country_cat_Chad', 'country_cat_DR Congo', 'country_cat_Dominican Republic', 'country_cat_Eswatini', 'country_cat_Fiji', 'country_cat_Gambia']

Dataset shape after encoding: (56721, 814)


## 6. Select Final Variables and Export

In [7]:
# Get the encoded categorical column names
encoded_cat_cols = onehot_encoder.get_feature_names_out(X_categorical_cols) if X_categorical_cols else []

# Final covariate list
final_X_cols = X_binary_cols + list(ordinal_mappings.keys()) + list(encoded_cat_cols)

# Select relevant columns
relevant_cols = Y_cols + T_cols + final_X_cols
mics_clean = mics[relevant_cols]

print(f"Final dataset shape: {mics_clean.shape}")
print(f"\nColumns in cleaned dataset:")
print(f"  - Outcomes: {len(Y_cols)}")
print(f"  - Treatment: {len(T_cols)}")
print(f"  - Covariates: {len(final_X_cols)}")
print(f"\nMissing values per column:")
print(mics_clean.isnull().sum()[mics_clean.isnull().sum() > 0])

Final dataset shape: (56721, 54)

Columns in cleaned dataset:
  - Outcomes: 2
  - Treatment: 1
  - Covariates: 51

Missing values per column:
PipedWater                     3
WellandSpringWater             3
RainandSurfaceWater            3
PurchasedWater                 3
Basic_water_service            2
Limited_water_service          2
Unimproved_water_service       3
Open_defecation             1209
dtype: int64


In [8]:
# Export cleaned data
mics_clean.to_csv("mics_clean.csv", index=False)
mics_clean.head()

,SomeRiskHome,VeryHighRiskHome,water_treatment,Any_U5,Girls_less_than15,Boys_15or_less,ImprovedWaterSource,PipedWater,WellandSpringWater,RainandSurfaceWater,...,country_cat_Togo,country_cat_Trinidad and Tobago,country_cat_Viet Nam,country_cat_Zimbabwe,WS1_g_Packaged/Bottled water,WS1_g_Piped water,WS1_g_Protected well/spring,WS1_g_Surface/Rain water,WS1_g_Tube/Well/Borehole,WS1_g_Unprotected well/spring
0,1,0,0,1,0,1,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,0,0,1,0,0,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,1,0,1,0,0,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,1,0,1,0,1,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,0,0,0,0,0,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
